In [9]:
%matplotlib inline

In [10]:
import pandas as pd
import numpy as np
from sys import stdout
import os, glob
from scipy.signal import detrend
from scipy import arctan2
import functions as f

In [11]:
import matplotlib.pyplot as plt

In [12]:
path_to_csvs   = "/Volumes/pdoubraw/repos/jet/data/TTU 200 m tower data/"
path_to_csvs   = "C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\"
filepaths_dict = f.get_file_paths(path_to_csvs, day=17)

In [13]:
height_feet_dict, height_meters_dict, header_names_dict, unique_heights, unique_heights_all = f.get_file_headers(path_to_csvs)

In [14]:
year   = 2017 
month  = 2
day    = 17
hour   = 18
minute = int('00')

In [15]:
desired_start = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)
desired_end   = desired_start + pd.Timedelta('10min')
timestamp_in_filename = desired_start.floor('30min')
print(timestamp_in_filename)

2017-02-17 18:00:00


In [16]:
filenames = sorted(glob.glob(os.path.join(path_to_csvs,'*{0:%Y%m%d}_T{0:%H%M}*'.format(timestamp_in_filename))))

# actual i/o

In [19]:
perturbations = {}
# Get Cosine Angles for each height 
cosAngles, boom_azm_deg, boom_vert_deg =f.get_boom_tower_angles(path_to_csvs) 


for filepath in filenames:

    group = int(os.path.split(filepath)[-1].split('TR_D')[-1].split('.csv')[0])

    print(filepath)

    df           = f.csv_to_df(filepath, header_names_dict[group])  
    subdf        = f.subset_df(df)
    df_dict      = f.df_to_df_dict(subdf)

    df_dict_rot  = f.rotate_df_dict(df_dict,cosAngles)
    df_dict_pert = f.compute_perturbations(df_dict_rot)

    for height in df_dict_pert.keys():

        perturbations[height] = df_dict_pert[height].loc[desired_start:desired_end].copy()

desired_index = pd.DatetimeIndex(start=desired_start, end=desired_end, freq='20ms').tz_localize('UTC')

for height in perturbations.keys():
    perturbations[height] = perturbations[height].reindex(desired_index).interpolate(method='index').iloc[0:30000].copy()  
    
print('Done!')

C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27892_D20170217_T1800_TR_D01.csv


TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]

# decide which key (distance above ground) we want

In [48]:
keys_feet = np.sort(list(perturbations.keys()))

In [49]:
keys_meters = np.sort(np.round([v*0.3048 for v in keys_feet],1))

In [50]:
keys_feet

array([  3.,   8.,  13.,  33.,  55., 155., 382., 519., 656.])

In [51]:
keys_meters

array([  0.9,   2.4,   4. ,  10.1,  16.8,  47.2, 116.4, 158.2, 199.9])

# grab subset of data desired

In [75]:
subset = perturbations[382][['U','V','W']]
subset.head()

,U,V,W
2017-02-17 18:00:00+00:00,-7.242492,-4.669800,0.455504
2017-02-17 18:00:00.020000+00:00,-7.235864,-4.695993,0.486649
2017-02-17 18:00:00.040000+00:00,-7.280556,-4.626015,0.532536
2017-02-17 18:00:00.060000+00:00,-7.155142,-4.534457,0.595691
2017-02-17 18:00:00.080000+00:00,-7.039852,-4.505760,0.640242


### rotate so that u is the main component

In [76]:
sonic_yaw   = np.arctan2(subset.V.mean(),subset.U.mean())
u1          = subset.U*np.cos(sonic_yaw)+subset.V*np.sin(sonic_yaw)
v1          = -subset.U*np.sin(sonic_yaw)+subset.V*np.cos(sonic_yaw)
w1          = subset.W.copy()

sonic_pitch = np.arctan2(w1.mean(),u1.mean())
u2          = u1*np.cos(sonic_pitch)+w1*np.sin(sonic_pitch)
v2          = v1.copy()
w2          = -u1*np.sin(sonic_pitch)+w1*np.cos(sonic_pitch)

print("PITCH: rotate by {0:.4f} deg".format(np.degrees(sonic_pitch)))
print("YAW: rotate by {0:.4f} deg".format(np.degrees(sonic_yaw)))

PITCH: rotate by 0.0307 deg
YAW: rotate by -144.7018 deg


In [77]:
subset.loc[:,'U'] = u2.copy()
subset.loc[:,'V'] = v2.copy()
subset.loc[:,'W'] = w2.copy()

### resample to same time step of turbsim simulations

In [78]:
subset = subset.resample('50ms').mean()
subset.head()

,U,V,W
2017-02-17 18:00:00+00:00,8.614786,-0.384506,0.486953
2017-02-17 18:00:00.050000+00:00,8.404855,-0.412054,0.613469
2017-02-17 18:00:00.100000+00:00,8.255492,-0.312492,0.611717
2017-02-17 18:00:00.150000+00:00,8.057799,-0.395673,0.381232
2017-02-17 18:00:00.200000+00:00,8.143530,0.037558,0.378368


In [60]:
subset.index = (subset.index-subset.index[0]).total_seconds()

In [61]:
subset

,U,V,W
0.00,8.614786,-0.384506,0.486953
0.05,8.404855,-0.412054,0.613469
0.10,8.255492,-0.312492,0.611717
0.15,8.057799,-0.395673,0.381232
0.20,8.143530,0.037558,0.378368
0.25,8.383487,0.276460,0.594283
0.30,8.610151,0.190134,0.546684
0.35,8.958748,0.004532,0.367888
0.40,9.014030,0.632273,0.287209
0.45,9.110663,1.188689,0.485760


### save for turbsim

In [37]:
fout = "../TIMESR_{0:%Y-%m-%d_%H-%M}_ROT.txt".format(desired_start)
subset.to_csv(fout,sep="\t",line_terminator='\n',float_format="%5.2f")

In [38]:
fout

'../TIMESR_2017-02-17_18-00_ROT.txt'